In [1]:
import re  # For preprocessing
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import pandas as pd
import os
os.listdir()
print(os.listdir("Data/"))#Contents of the Data folder

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

['cz_item_category', 'item_content_cz', 'item_content_filtered (with_duplicates).csv', 'item_content_filtered_no_duplicates.csv', 'item_content_viewed_sid.csv', 'item_cz', 'item_list.csv', 'list_category', 'material_attributes_cz', 'product_views_cz', 'product_views_cz_filtered.csv', 'product_views_unified.csv', 'sentences.csv']


In [2]:
dataFile = "Data/product_views_unified.csv"
number_rows = 30000000
data_2020 = pd.read_csv(dataFile,nrows=number_rows)
print(data_2020)

         user_id_anon    product_id
0            14723304  100011803451
1            14723304  100011612041
2            14723304  100016624290
3            14723304       1093107
4            14723304  100016287573
...               ...           ...
2999995      57137615       1489324
2999996      57137615       1285712
2999997      57137615       1489324
2999998      57137615       1399946
2999999      57137615       1400006

[3000000 rows x 2 columns]


In [3]:
data_2020['product_id'] = data_2020.loc[:,'product_id'].map(str)
print(data_2020)

         user_id_anon    product_id
0            14723304  100011803451
1            14723304  100011612041
2            14723304  100016624290
3            14723304       1093107
4            14723304  100016287573
...               ...           ...
2999995      57137615       1489324
2999996      57137615       1285712
2999997      57137615       1489324
2999998      57137615       1399946
2999999      57137615       1400006

[3000000 rows x 2 columns]


In [4]:
sentences = data_2020.groupby('user_id_anon')['product_id'].apply(list)
print(sentences)

user_id_anon
645         [655028, 1338684, 100018643178, 100018644575, ...
788         [100016425290, 452568001, 1086821, 1403084, 14...
871         [551806, 1305893, 1293084, 1389660, 1510064, 1...
1345        [100018205007, 100018206462, 100018206445, 100...
2981        [1480352, 1419661, 1105542, 1419654, 666213, 1...
                                  ...                        
62563472    [1349506, 1349510, 1349506, 1397954, 1490283, ...
62563934    [100016141442, 100016143151, 100016145005, 100...
62565140    [1236035, 100002172473, 645720, 100002786928, ...
62565992    [1339130, 1330960002, 1330960002, 1330960002, ...
62570516    [1327813005, 1327813005, 673184004, 1455333, 1...
Name: product_id, Length: 62897, dtype: object


In [36]:
most_viewed_element = data_2020["product_id"].value_counts().idxmax()
print(most_viewed_element)

100024602647


In [5]:
import multiprocessing

from gensim.models import Word2Vec
import numpy as np

cores = multiprocessing.cpu_count() # Count the number of cores in a computer

# w2v_model = Word2Vec(min_count=10,
#                      window=2,
#                      size=300,
#                      sample=6e-5, 
#                      alpha=0.05, 
#                      min_alpha=0.001, 
#                      negative=20,
#                      workers=cores-1)

In [11]:
dataFile = "Data/product_views_unified.csv"
test_rows = 31502086 - number_rows
test_data = pd.read_csv(dataFile,names=["user_id_anon","product_id"],skiprows=number_rows,nrows=1000000)
test_data['product_id'] = test_data.loc[:,'product_id'].map(str)
test_set = test_data.groupby('user_id_anon')['product_id'].apply(list)
print(test_set)

user_id_anon
3349        [100019718522, 100019718522, 1478497, 10001971...
50382       [871204004, 1000972002, 1423921, 1209959, 1381...
99667       [100034162591, 100016262061, 100016270803, 100...
109964      [100017557065, 666619, 1318349, 1299609, 13539...
111780      [772945, 1499435, 867627, 100017832546, 105519...
                                  ...                        
62474251    [100017149736, 100017149736, 100017149736, 100...
62514848    [618966, 1498611, 1341309, 100004989623, 14986...
62536557    [536577, 1397763, 100023888588, 100018528628, ...
62554666    [1455389, 100008723422, 1455389, 1120664, 1395...
62558727    [1503117001, 100018855245, 100004385793, 10002...
Name: product_id, Length: 2135, dtype: object


In [19]:
import itertools

parm_dict = {'size':(100,200,300),'window':(2,5),'min_count':(5,10),'alpha':(0.03,0.02,0.01)}

def cust_param_search(parm_dict,sentences,test_set):
    score_best, parm_best = 0,()
   
    size,window, min_count, alpha = [tup for k,tup in parm_dict.items()] # Individual parm tuples

    parm_combo = list(itertools.product(size,window, min_count, alpha)) # Create all combinations

    for parms in parm_combo:
        s, w, m, a = parms
        print (parms)
        model = Word2Vec(min_count=m,
                         window=w,
                         size=s,
                         alpha=a,
                         workers=cores-1)
        model.build_vocab(sentences, progress_per=5000)
        model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

        model.init_sims(replace=True)
        precision = []
        for i in test_set: #SCORE FUNCTION pro nejvíce viděný produkt oproti všem ostatním
            test = i
            k=0
            num_history = 5
            while k < len(test):
                    if test[k] not in model.wv.vocab:
                        test.pop(k)
                    else:
                        k=k+1
            if len(test) > num_history:        
                vector = model.wv.most_similar(positive=test[:num_history],topn=len(test)-num_history)
                vector = [l for (l,n) in vector]
                num = 0
                for l in range(len(vector)):
                    if vector[l] in test[num_history:]:
                        num = num + 1
#                         print("true positive")
#                         print("position " + str(l) + " of " + str(len(vector)) + " positions")
                precision.append(num / (len(test)-num_history))
            #         print(precision)
        
        score = sum(precision)/len(precision)
        print(score)

        if score > score_best:
            score_best = score
            parm_best = parms          
    print("Best score -",score_best, "Best parms - ",parm_best)

In [20]:
cust_param_search(parm_dict,sentences,test_set)

INFO - 16:49:06: collecting all words and their counts
INFO - 16:49:06: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:49:06: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types
INFO - 16:49:06: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types


(100, 2, 5, 0.03)


INFO - 16:49:06: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 16:49:06: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 16:49:07: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 16:49:07: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 16:49:07: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 16:49:07: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 16:49:07: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 16:49:07: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 16:49:07: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 16:49:07: PROGRESS: at sentence #60000, processed 2862554 words, keeping 351953 word types
INFO - 16:49:07: colle

position 38 of 41 positions
position 16 of 33 positions
position 32 of 62 positions
position 0 of 73 positions
position 7 of 14 positions
position 10 of 23 positions
position 0 of 26 positions
position 1 of 26 positions
position 7 of 26 positions
position 13 of 39 positions
position 9 of 90 positions
position 42 of 90 positions
position 16 of 53 positions
position 25 of 53 positions
position 0 of 21 positions
position 11 of 30 positions
position 3 of 20 positions
position 4 of 20 positions
position 2 of 56 positions
position 19 of 56 positions
position 81 of 172 positions
position 157 of 172 positions
position 160 of 172 positions
position 168 of 172 positions
position 0 of 68 positions
position 2 of 68 positions
position 20 of 68 positions
position 31 of 68 positions
position 66 of 68 positions
position 53 of 432 positions
position 1 of 53 positions
position 25 of 53 positions
position 36 of 53 positions
position 44 of 53 positions
position 51 of 53 positions
position 1 of 92 position

INFO - 16:51:49: collecting all words and their counts
INFO - 16:51:49: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:51:49: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types
INFO - 16:51:49: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types


...

INFO - 16:59:47: collecting all words and their counts
INFO - 16:59:47: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:59:47: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types



position 0 of 55 positions
position 41 of 55 positions
position 0 of 44 positions
position 11 of 44 positions
position 0 of 42 positions
position 11 of 16 positions
0.016877908926872146
(100, 2, 10, 0.02)


INFO - 16:59:47: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 16:59:47: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 16:59:47: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 16:59:47: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 16:59:48: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 16:59:48: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 16:59:48: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 16:59:48: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 16:59:48: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 16:59:48: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 16:59:48: PROGRE

position 8 of 37 positions
position 10 of 59 positions
position 12 of 59 positions
position 2 of 63 positions
position 0 of 16 positions
position 11 of 16 positions
position 1 of 26 positions
position 11 of 26 positions
position 11 of 34 positions
position 30 of 34 positions
position 9 of 88 positions
position 12 of 88 positions
position 48 of 88 positions
position 70 of 88 positions
position 17 of 51 positions
position 0 of 19 positions
position 17 of 30 positions
position 1 of 20 positions
position 2 of 20 positions
position 2 of 56 positions
position 4 of 56 positions
position 47 of 145 positions
position 49 of 145 positions
position 88 of 145 positions
position 99 of 145 positions
position 123 of 145 positions
position 0 of 53 positions
position 1 of 53 positions
position 15 of 53 positions
position 16 of 53 positions
position 20 of 53 positions
position 46 of 53 positions
position 14 of 426 positions
position 306 of 426 positions
position 21 of 36 positions
position 6 of 47 positi

KeyboardInterrupt: 

In [39]:
#S JINOU SCORING FUNKCI

# import itertools

# parm_dict = {'size':(100,200,300),'window':(2,5),'min_count':(5,10),'alpha':(0.03,0.02,0.01)}

# def cust_param_search(parm_dict,sentences,test_set,most_viewed_element):
#     score_best, parm_best = 0,()
   
#     size,window, min_count, alpha = [tup for k,tup in parm_dict.items()] # Individual parm tuples

#     parm_combo = list(itertools.product(size,window, min_count, alpha)) # Create all combinations

#     for parms in parm_combo:
#         s, w, m, a = parms
#         print (parms)
#         model = Word2Vec(min_count=m,
#                          window=w,
#                          size=s,
#                          alpha=a,
#                          workers=cores-1)
#         model.build_vocab(sentences, progress_per=5000)
#         model.train(sentences, total_examples=model.corpus_count, epochs=10, report_delay=1)

#         model.init_sims(replace=True)
#         similarity = []
#         for i in test_set: #SCORE FUNCTION pro nejvíce viděný produkt oproti všem ostatním
#             simil = []
#             for j in i:
#                 if j in model.wv.vocab and j != most_viewed_element:
# #                     print(model.wv.similarity(most_viewed_element,j))
#                     simil.append(model.wv.similarity(most_viewed_element,j))
#             if len(simil)>0:
#                 similarity.append(sum(simil)/len(simil))
        
#         score = sum(similarity)/len(similarity)
#         print(score)

#         if score > score_best:
#             score_best = score
#             parm_best = parms          
#     print("Best score -",score_best, "Best parms - ",parm_best)

In [16]:
# model = Word2Vec(min_count=5,
#                  window=5,
#                  size=200,
#                  alpha=0.025,
#                  workers=cores-1)
# model.build_vocab(sentences, progress_per=5000)
# model.train(sentences, total_examples=model.corpus_count, epochs=20, report_delay=1)

# model.init_sims(replace=True)
# precision = []
# for i in test_set: #SCORE FUNCTION pro nejvíce viděný produkt oproti všem ostatním
#     test = i
#     k=0
#     num_history = 5
#     while k < len(test):
#             if test[k] not in model.wv.vocab:
#                 test.pop(k)
#             else:
#                 k=k+1
#     if len(test) > num_history:        
#         vector = model.wv.most_similar(positive=test[:num_history],topn=len(test)-num_history)
#         vector = [l for (l,n) in vector]
#         num = 0
#         for l in range(len(vector)):
#             if vector[l] in test[num_history:]:
#                 num = num + 1
#                 print("true positive")
#                 print("position " + str(l) + " of " + str(len(vector)) + " positions")
#         precision.append(num / (len(test)-num_history))
# #         print(precision)

# score = sum(precision)/len(precision)
# print(score)

INFO - 16:40:28: collecting all words and their counts
INFO - 16:40:28: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:40:28: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types
INFO - 16:40:28: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 16:40:28: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 16:40:28: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 16:40:29: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 16:40:29: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 16:40:29: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 16:40:29: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 16:40:29: PROGRESS: at sentence #45000, processed 2146102 words, keeping 3077

true positive
position 17 of 41 positions
true positive
position 9 of 34 positions
true positive
position 13 of 33 positions
true positive
position 1 of 62 positions
true positive
position 11 of 62 positions
true positive
position 33 of 62 positions
true positive
position 72 of 73 positions
true positive
position 19 of 23 positions
true positive
position 20 of 21 positions
true positive
position 2 of 26 positions
true positive
position 6 of 26 positions
true positive
position 2 of 39 positions
true positive
position 19 of 39 positions
true positive
position 21 of 90 positions
true positive
position 58 of 90 positions
true positive
position 65 of 90 positions
true positive
position 82 of 90 positions
true positive
position 87 of 90 positions
true positive
position 30 of 53 positions
true positive
position 0 of 21 positions
true positive
position 0 of 20 positions
true positive
position 2 of 20 positions
true positive
position 14 of 20 positions
true positive
position 8 of 56 positions
t